# self attention 

In [ ]:
import torch
import torch.nn as nn
import math
class muti_head_attention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.dropout = nn.Dropout(dropout)
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.out = nn.Linear(d_model, d_model)
    def forward(self, X, mask=None):
        batch_size, seuqence_length, d_model = X.size()
        # (batch_size, seuqence_length, d_model) -> (batch_size, seuqence_length, num_heads, d_k) -> (batch_size, num_heads, seuqence_length, d_k)
        K = self.k_linear(X)
        Q = self.q_linear(X)
        V = self.v_linear(X)
        k_state = K.view(batch_size, seuqence_length, self.num_heads, self.d_k).transpose(1, 2)
        q_state = Q.view(batch_size, seuqence_length, self.num_heads, self.d_k).transpose(1, 2)
        v_state = V.view(batch_size, seuqence_length, self.num_heads, self.d_k).transpose(1, 2)
        # (batch_size, num_heads, sequence_length, sequence_length)
        attention_weight = q_state @ k_state.transpose(1, 2) / math.sqrt(self.d_k)
        if mask is not None:
            attention_weight = attention_weight.masked_fill(
                mask == 0,
                float("inf")
            )
        attention_weight = torch.softmax(
            attention_weight,
            dim = -1 # 在最后一个维度上进行softmax
        )
        attention_weight = self.dropout(attention_weight)
        # (batch_size, num_heads, sequence_length, d_k)
        attention_value = attention_weight @ v_state
        # (batch_size, num_heads, sequence_length, d_k) -> (batch_size, sequence_length, num_heads, d_k) -> (batch_size, sequence_length, d_model)
        attention_value = attention_value.transpose(1,2).contiguous()
        attention_value = attention_value.view(batch_size, seuqence_length, -1)
        output = self.out(attention_value)
        return output
class feed_forward(nn.Module): # 前馈神经网络
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, X):
        X = torch.relu(self.linear1(X))
        X = self.dropout(X)
        X = self.linear2(X)
        return X

# muti-head

In [ ]:
IM

In [ ]:
import torch
import torch.nn as nn 
import math

class SelfAttention(nn.Module):
    def __init__(self, d_model, heads, dropout_rate = 0.1):
        super().__init__()
        self.d_model = d_model
        self.heads = heads
        self.head_dim = d_model // heads
        self.Q_projetion = nn.Linear(d_model, d_model) #(d_mode, head_dim * heads)
        self.K_projetion = nn.Linear(d_model, d_model)
        self.V_projetion = nn.Linear(d_model, d_model)
        self.out_projetion = nn.Linear(d_model, d_model)
        self.attention_dropout = nn.Dropout(dropout_rate)

    def forward(self, X, attention_mask = None):
        # shape (batch_size, seq_len, d_model)
        batch, seq , _ = X.size()
        Q = self.Q_projetion(X)
        K = self.K_projetion(X)
        V = self.V_projetion(X)

        # (b, s, d) ==> (b, heads, s, head_dim) # d = heads * head_dim 
        q_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        k_state = K.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        v_state = V.view(batch, seq, self.heads, self.head_dim).transpose(1,2)

        # shape (b, s, s)
        # (b, heads, s, head_dim) ==> (b,  head_dim, s)
        # shape (b, heads,s, s)
        attention_weight = q_state @ k_state.transpose(-1, -2) / math.sqrt(self.head_dim)
        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        print(attention_weight.shape)
        attention_weight = torch.softmax(
            attention_weight,
            dim = -1
        )

        attention_weight = self.attention_dropout(attention_weight)
        # (b, heads, s, head_dim)
        attention_value = attention_weight @ v_state
        out = attention_value.transpose(1, 2).contiguous()
        out = out.view(batch, seq, -1)
        out = self.out_projetion
        return out

        return attention_value
X = torch.rand(3, 4, 2)
mask = torch.tensor(
    [
        [1,1,1,0],
        [1,1,0,0],
        [1,0,0,0]
    ]
)
mask = mask.unsqueeze(1).repeat(1,4,1)
net = SelfAttention(2,1)
net(X, attention_mask = mask)

In [ ]:
class Mutiheadattention(nn.Module):
    def __init__(self, d_model, heads, gropout_rate):
        super().__init__()
        self.d_model = d_model
        self.heads = heads
        self.head_dim = d_model // heads

        self.Q_proj = nn.Linear(d_model, d_model)
        self.K_proj = nn.Linear(d_model, d_model)
        self.V_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)

        self.attention_dropout = nn.Dropout(gropout_rate)
    def forward(self, X, attention_mask = None):
        # (batch, seq, d_model)
        batch, seq, _ = X.size()
        Q = self.Q_proj(X)
        K = self.K_proj(X)
        V = self.V_proj(X)
        out = self.out_proj(X)

        # (batch, seq, d_model) ==> (b, heads, s, head_dim)
        q_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        k_state = K.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        v_state = V.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        # (b, heads, s, s)
        attention_weight = q_state @ v_state.transpose(-1, -2) / math.sqrt(self.head_dim)

        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        
        attention_weight = torch.softmax(
            attention_weight, 
            dim = -1
        )

        attention_weight = self.attention_dropout(attention_weight)

        attention_value = attention_weight @ v_state
        # (b, heads, s, head_dim) ==> (batch, seq, heads, head_dim)
        out = attention_value.transpose(1,2).contiguous()
        # (batch, seq, heads, head_dim) ==> (batch, seq, d_model)
        out = out.view(batch, seq, -1)
        out = self.out_proj
        return out
        

In [ ]:
import torch
import torch.nn as nn 
import math
class Mutiattention(nn.Module):
    def __init__(self, heads, d_model, dropout_rate):
        super().__init__()
        self.heads = heads
        self.d_model = d_model
        self.head_dim = self.d_model // self.heads
        self.Q_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.K_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.V_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.out_proj = nn.Linear(d_model2, d_model) # (d_model, heads * head_dim)
        self.attention_dropout = nn.Dropout(dropout_rate)

    def forward(self, X, attention_mask = None):
        # shape (batch_size, seq_len, d_model)
        batch, seq, _ = X.size()
        Q = self.Q_proj(X)
        K = self.K_proj(X)
        V = self.V_proj(X)

        # shape (b, heads, seq, head_dim)
        q_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        k_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        v_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        # shape(b , heads, seq, seq)
        attention_weight = q_state @ k_state.transpose(-2,-1) // math.sqrt(self.head_dim)
        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        attention_weight = torch.softmax(
            attention_weight, dim = -1
        )

        attention_weight = self.attention_dropout(attention_weight)
        # shape(batch, heads, seq, head_dim)
        attention_value = attention_weight @ v_state
        # shape(b, s, heads, head_dim)
        out = attention_value.tranpose(1,2),contiguous()
        out = out.view(batch, seq, -1)
        output = self.out_proj(out)
        return output


In [ ]:
import torch
import torch.nn as nn
import math
class Group_attention(nn.Module):
    def __init__(self, heads, d_model,numers_key_value, dropout_rate):
        super().__init__()
        assert d_model % heads == 0
        assert heads % numers_key_value == 0
        self.d_model = d_model
        self.heads = heads
        self.head_dim = self.d_model // heads
        self.numers_key_value = numers_key_value #self.heads // 4

        self.Q_proj = nn.Linear(d_model, heads * self.head_dim) 
        self.K_proj = nn.Linear(d_model, numers_key_value * self.head_dim) 
        self.V_proj = nn.Linear(d_model, numers_key_value * self.head_dim) 
        self.out_proj = nn.Linear(d_model, d_model) 

        self.attention_dropout = nn.Dropout(dropout_rate)
    def forward(self, X, attention_mask = None):
        # X shape(batch, seq, d_model)
        batch_size, seq_len, _ = X.size()
        Q = self.Q_proj(X)
        K = self.K_proj(X)
        V = self.V_proj(X)

        # atttetntion_weight (batch, heads, seq_q, seq_k)
        q = Q.view(batch_size, seq_len, self.heads, self.head_dim).transpose(1,2)
        k = K.view(batch_size, seq_len, self.numers_key_value, self.head_dim).transpose(1,2)
        v = V.view(batch_size, seq_len, self.numers_key_value, self.head_dim).transpose(1,2)
        # k, v 需要repeat。torch 存在广播操作。
        k = k.repeat_interleave(self.heads // self.numers_key_value, dim = 1)
        v = v.repeat_interleave(self.heads // self.numers_key_value, dim = 1)

        attention_weight = q @ k.transpose(-1,-2) / math.sqrt(self.head_dim)
        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        attention_weight = torch.softmax(attention_weight, dim = -1)
        attention_weight = self.attention_dropout(attention_weight)
        output = attention_weight @ v
        return output


In [ ]:
import torch
import torch.nn as nn
import math 
class attention(nn.Module):
    def __init__(self, heads, hidden_dim, dropout_rate):
        super().__init__()
        assert hidden_dim % heads = 0
        self.hidden_dim = hidden_dim
        self.heads = heads
        self.head_dim = hidden_dim // heads
        self.Q = nn.Linear(hidden_dim, hidden_dim) # (hidden_dim, head * head_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim) # (hidden_dim, head * head_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim) # (hidden_dim, head * head_dim)
        self.O = nn.Linear(hidden_dim, hidden_dim) # (hidden_dim, head * head_dim)
        self.attention_dropout = dropout_rate
    def forward(self, X, attention_mask = None):
        #X : (batch_size, seq_len, hiddem_dim)
        batch_size, seq_len, hiddem_dim = X.size()
        Q = self.Q(X)
        K = self.K(X)
        V = self.V(X)
        #分割头
        #(batch, heads, seq, head_dim)
        q_state = Q.view(batch_size, seq_len, self.heads, self.head_dim).shape(1,2)
        k_state = K.view(batch_size, seq_len, self.heads, self.head_dim).shape(1,2)
        v_state = V.view(batch_size, seq_len, self.heads, self.head_dim).shape(1,2)
        #(b,hs,s,s)
        attention_weight = q_state @ k_state.tranpose(-1,-2) // math.sqrt(self.head_dim)
        if attention_mask:
            attention_weight = attention_weight.masked_fill(
                attention_mask = 0,
                float('-inf')
            )
        attention_weight = torch.softmax(
            attention_weight,
            dim = -1
        )
        attention_weight = self.attention_dropout(
            attention_weight
        )
        #(b,hs, s, head_dim)
        attention_value = attention_weight @ v_state
        #(b, s, heads, head_dim)
        attention_value.tranpose(1,2),contiguous()
        out = out.view(batch_size, seq_len, -1)
        output = self.O(out)
        return output

In [ ]:
import torch
import torch.nn as nn 
import math
class Mutiattention(nn.Module):
    def __init__(self, heads, d_model, dropout_rate):
        super().__init__()
        self.heads = heads
        self.d_model = d_model
        self.head_dim = self.d_model // self.heads
        self.Q_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.K_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.V_proj = nn.Linear(d_model, d_model2) # (d_model, heads * head_dim)
        self.out_proj = nn.Linear(d_model2, d_model) # (d_model, heads * head_dim)
        self.attention_dropout = nn.Dropout(dropout_rate)

    def forward(self, X, attention_mask = None):
        # shape (batch_size, seq_len, d_model)
        batch, seq, _ = X.size()
        Q = self.Q_proj(X)
        K = self.K_proj(X)
        V = self.V_proj(X)

        # shape (b, heads, seq, head_dim)
        q_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        k_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        v_state = Q.view(batch, seq, self.heads, self.head_dim).transpose(1,2)
        # shape(b , heads, seq, seq)
        attention_weight = q_state @ k_state.transpose(-2,-1) // math.sqrt(self.head_dim)
        if attention_mask is not None:
            attention_weight = attention_weight.masked_fill(
                attention_mask == 0,
                float('-inf')
            )
        attention_weight = torch.softmax(
            attention_weight, dim = -1
        )

        attention_weight = self.attention_dropout(attention_weight)
        # shape(batch, heads, seq, head_dim)
        attention_value = attention_weight @ v_state
        # shape(b, s, heads, head_dim)
        out = attention_value.tranpose(1,2),contiguous()
        out = out.view(batch, seq, -1)
        output = self.out_proj(out)
        return output
